# 0 Setting

In [33]:
# Parameter Setting
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
folder_name = 'Research-(D5) Synthesized input model'

config = {
    'learning_rate': 1e-4,
    'batch_size': 32,
    'seq_length': 5,

    'shuffle': False,
    'criterion': torch.nn.CrossEntropyLoss(),
    'seed': 42,
    'valid_ratio': 0.2,
    'test_ratio': 0.2,
    'max_length': 512,
    'n_epochs': 3000,
    'early_stop': 100,
}

feature = [
    # X_1
    # 'input_ids',
    # 'attention_mask',
    # 'section_dummy',
    # 'publication_dummy',

    # X_2
    # 1. tech indicator
    # 'Open',
    # 'High',
    # 'Low',
    # 'Close',
    # 'Volume',
    # 'Dividends',
    # 'Stock Splits',
    'today_return',
    # 'today_return_cate',
    # 'Sma',
    # 'Rsi',
    # 'Kd',
    # 'Ema_12',
    # 'Ema_26',
    # 'Macd',
    'sentiment',

    # 2. market index
    '^DJI',
    '^GSPC',
    '^NDX',
    '^IXIC',
    '^SOX',

    # y
    # '1_day_return',
    # '2_day_return',
    # '3_day_return',
    # '4_day_return',
    # '5_day_return',
    # '1_day_return_cate',
    # '2_day_return_cate',
    # '3_day_return_cate',
    # '4_day_return_cate',
    # '5_day_return_cate',
    # '^DJI', '^DJI_1_day_return', '^GSPC', '^GSPC_1_day_return',
    #    '^NDX', '^NDX_1_day_return', '^IXIC', '^IXIC_1_day_return', '^SOX',
    #    '^SOX_1_day_return',
    # 'excess_return_^DJI',
    # 'excess_return_^DJI_cate',
    # 'excess_return_^GSPC',
    'excess_return_^GSPC_cate',
    # 'excess_return_^NDX',
    # 'excess_return_^NDX_cate',
    # 'excess_return_^IXIC',
    # 'excess_return_^IXIC_cate',
    # 'excess_return_^SOX',
    # 'excess_return_^SOX_cate',


    # Do not mark the datetime, it's for operation
    'datetime',
    ]

# All the news dataset
# time_start = '2016-01-01T00:00:00'
# time_end = '2020-04-02T00:00:00'

time_start = '2016-01-01T00:00:00'
time_end = '2019-12-31T00:00:00'

print(len(feature)-2)

company_list = [
    "Information Technology",
    "Health Care",
    "Financials",
    "Industrials",
    "Consumer Discretionary",
    "Energy",
    "Materials",
    "Communication Services",
    "Utilities",
    "Real Estate",
    "Consumer Staples"
]

process_id = 0  #26

company_name = company_list[process_id]
config_2 = {'input_path': '/content/drive/MyDrive/Colab Notebooks/'+folder_name+'/data/2_'+company_name+'_for_model 0.05.csv',
            # 'save_path': '/content/drive/MyDrive/Colab Notebooks/'+folder_name+'/model_saved/model.ckpt',
            }

7


## (1) Import

In [34]:
# Google
from google.colab import drive
drive.mount('/content/drive')

# pip installation
!pip install transformers

# Basic
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

# Sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# PyTorch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter

from transformers import XLNetModel, XLNetTokenizer, BertTokenizer, BertModel

# others
from datetime import datetime, timedelta
from tqdm import tqdm
from torchsummary import summary
import ast

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [35]:
def same_seed(seed):
    '''Fixes random number generator seeds for reproducibility.'''
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

# Set seed for reproducibility
same_seed(config['seed'])


In [36]:
df = pd.read_csv(config_2['input_path'])
df = df.sort_values(by='datetime', ascending=True)
df

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains,today_return,today_return_cate,...,excess_return_^IXIC_cate,excess_return_^SOX,excess_return_^SOX_cate,excess_return_^NYA,excess_return_^NYA_cate,input_ids,attention_mask,section_dummy,publication_dummy,sentiment
0,38.157610,38.357196,37.749361,38.348125,21638600,0.0,0.0,0.0,0.004993,1,...,0,0.006342,1,-0.007764,0,"[[101, 18817, 1005, 1055, 8292, 2015, 2355, 25...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[0, 0, 0, 0, 0, 0, 0, 0, 1], [1, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, ...",0.486591
1,38.466063,38.556784,38.075959,38.248329,16067200,0.0,0.0,0.0,-0.005660,0,...,0,0.013500,1,0.006355,1,"[[101, 13938, 1005, 1055, 8292, 2015, 2355, 25...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [1, ...",0.479906
2,37.685864,38.030605,37.549780,37.776585,13858400,0.0,0.0,0.0,0.002407,1,...,1,0.001351,1,0.001034,1,"[[101, 2115, 9432, 27918, 1024, 3795, 15768, 1...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, ...",0.480993
3,37.005450,37.486274,36.642561,36.660706,16840700,0.0,0.0,0.0,-0.009316,0,...,1,0.003363,1,0.000485,1,"[[101, 1996, 3316, 2008, 3627, 4684, 2024, 263...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, ...",0.482691
4,36.978211,37.105225,36.325016,36.370377,19233100,0.0,0.0,0.0,-0.016438,0,...,1,0.003958,1,0.004467,1,"[[101, 15768, 2485, 3816, 13463, 3514, 25912, ...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, ...","[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, ...",0.479977
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1065,77.318843,78.772933,75.855056,76.184647,29242700,0.0,0.0,0.0,-0.014669,0,...,1,0.002589,1,0.009174,1,"[[101, 6207, 7283, 5599, 7276, 5126, 2147, 200...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, ...",0.484674
1066,77.202517,79.577528,76.960168,79.393349,23740100,0.0,0.0,0.0,0.028378,1,...,0,0.004554,1,-0.008134,0,"[[101, 2833, 4425, 15508, 6617, 1999, 2149, 20...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, ...","[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, ...",0.478539
1067,79.209155,80.634165,77.503021,77.910172,19994400,0.0,0.0,0.0,-0.016399,0,...,0,0.005237,1,-0.006226,0,"[[101, 21887, 23350, 20861, 1999, 8244, 1521, ...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, ...",0.503434
1068,75.215239,76.824436,73.644826,74.197380,27492400,0.0,0.0,0.0,-0.013533,0,...,1,-0.013970,0,0.001834,1,"[[101, 21887, 23350, 2444, 14409, 1024, 1057, ...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, ...",0.354164


In [37]:
df.publication_dummy[0]

'[[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]'

In [38]:
df.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits',
       'Capital Gains', 'today_return', 'today_return_cate', '1_day_return',
       '2_day_return', '3_day_return', '4_day_return', '5_day_return',
       '1_day_return_cate', '2_day_return_cate', '3_day_return_cate',
       '4_day_return_cate', '5_day_return_cate', 'Sma', 'Rsi', 'Kd', 'Ema_12',
       'Ema_26', 'Macd', 'datetime', '^DJI', '^DJI_1_day_return', '^GSPC',
       '^GSPC_1_day_return', '^NDX', '^NDX_1_day_return', '^IXIC',
       '^IXIC_1_day_return', '^SOX', '^SOX_1_day_return', '^NYA',
       '^NYA_1_day_return', 'excess_return_^DJI', 'excess_return_^DJI_cate',
       'excess_return_^GSPC', 'excess_return_^GSPC_cate', 'excess_return_^NDX',
       'excess_return_^NDX_cate', 'excess_return_^IXIC',
       'excess_return_^IXIC_cate', 'excess_return_^SOX',
       'excess_return_^SOX_cate', 'excess_return_^NYA',
       'excess_return_^NYA_cate', 'input_ids', 'attention_mask',
       'section_dummy', '

In [39]:
# Only contain selected features
df = df[feature]
df.columns

Index(['today_return', 'sentiment', '^DJI', '^GSPC', '^NDX', '^IXIC', '^SOX',
       'excess_return_^GSPC_cate', 'datetime'],
      dtype='object')

## check data

In [40]:
df.isnull().sum()

today_return                0
sentiment                   0
^DJI                        0
^GSPC                       0
^NDX                        0
^IXIC                       0
^SOX                        0
excess_return_^GSPC_cate    0
datetime                    0
dtype: int64

In [41]:

df = df.dropna()
df = df.reset_index(drop=True)
df.isnull().sum()

today_return                0
sentiment                   0
^DJI                        0
^GSPC                       0
^NDX                        0
^IXIC                       0
^SOX                        0
excess_return_^GSPC_cate    0
datetime                    0
dtype: int64

## (2) Time Period Selection

In [42]:
# We use index to filter for time periods
df = df[(df['datetime']> time_start) & (df['datetime'] < time_end)]

# Drop datetime after using it
df.drop(columns=['datetime'], inplace=True)
df.shape

<ipython-input-42-c71406b1a572>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['datetime'], inplace=True)


(1006, 8)

## (3) Transform str back to list

In [43]:
df.isnull().sum()

today_return                0
sentiment                   0
^DJI                        0
^GSPC                       0
^NDX                        0
^IXIC                       0
^SOX                        0
excess_return_^GSPC_cate    0
dtype: int64

In [44]:
# # 将字符串转换回列表的函数
# def string_to_list(s):
#     return ast.literal_eval(s)

# # 将列中的字符串转换回列表
# df['input_ids'] = df['input_ids'].apply(string_to_list)
# df['attention_mask'] = df['attention_mask'].apply(string_to_list)
# df['section_dummy'] = df['section_dummy'].apply(string_to_list)
# df['publication_dummy'] = df['publication_dummy'].apply(string_to_list)

## (4) Train_test_split

In [45]:
# 1. Set up X, y
to_remove_list = ['datetime', 'excess_return_^GSPC_cate']

# Filter out values in to_remove_list
filtered_list = [x for x in feature if x not in to_remove_list]

X = df[filtered_list]
y = df['excess_return_^GSPC_cate']



# # 1. Set up X, y
# to_remove_list = ['datetime', '1_day_return_cate']

# # Filter out values in to_remove_list
# filtered_list = [x for x in feature if x not in to_remove_list]

# X = df[filtered_list]
# y = df['1_day_return_cate']

In [46]:
# print(X['section_dummy'])
# print(type(X['section_dummy'][0]))
# temp_array = np.array(X['section_dummy'][0])
# print(temp_array.dtype)

# def int_to_float(int_list):
#   float_list = np.array(int_list, dtype=np.float32)
#   return float_list
# X['section_dummy'] = X['section_dummy'].apply(int_to_float)
# X['publication_dummy'] = X['publication_dummy'].apply(int_to_float)\
# temp_array = np.array(X['section_dummy'][0])
# print(temp_array.dtype)


In [47]:
# def int_to_float(int_list):
#   float_list = np.array(int_list, dtype=np.float32)
#   return float_list
# X['section_dummy'] = X['section_dummy'].apply(int_to_float)
# X['publication_dummy'] = X['publication_dummy'].apply(int_to_float)

In [48]:
# Check X, y shape
print('X:', X.shape)
print('y:', y.shape)

X: (1006, 7)
y: (1006,)


In [49]:
# 2. train_test_split
# val dataset for final examination
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=config['seed'], shuffle=config['shuffle'])
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=config['seed'], shuffle=config['shuffle'])
X_train


,today_return,sentiment,^DJI,^GSPC,^NDX,^IXIC,^SOX
0,0.004993,0.486591,-0.014739,-0.012531,0.002854,0.001111,0.008979
1,-0.005660,0.479906,0.000651,0.001455,-0.006175,-0.005370,-0.012003
2,0.002407,0.480993,-0.014475,-0.010663,0.007758,0.004570,-0.011092
3,-0.009316,0.482691,-0.022161,-0.021271,-0.009984,-0.009917,-0.010667
4,-0.016438,0.479977,-0.010456,-0.012302,-0.015069,-0.016601,-0.019801
...,...,...,...,...,...,...,...
598,-0.000870,0.484560,0.000298,-0.001612,-0.001512,-0.001358,-0.003253
599,-0.000432,0.545957,0.005234,-0.000870,-0.002137,-0.001661,-0.004453
600,-0.006303,0.494470,-0.008509,-0.005076,-0.005924,-0.005712,-0.001986
601,0.015698,0.490091,0.005215,0.007115,0.015533,0.012397,0.015664


## (5) Scaler

In [50]:
scale_feature = [
    # X_2
    # 1. tech indicator
    # 'Open',
    # 'High',
    # 'Low',
    # 'Close',
    # 'Volume',
    # 'Dividends',
    # 'Stock Splits',
    'today_return',
    # 'Today_trend_cate',
    # 'Sma',
    # 'Rsi',
    # 'Kd',
    # 'Ema_12',
    # 'Ema_26',
    # 'Macd',
    'sentiment',

    # 2. market index
    '^DJI',
    '^GSPC',
    '^NDX',
    '^IXIC',
    '^SOX',
    # 'datetime'
    ]

def CustomScaler(X_train, X_val, X_test):
  scaler = MinMaxScaler()
  for i in scale_feature:

    # 對特定欄位進行標準化
    X_train_scaled = scaler.fit_transform(X_train[[i]])
    X_val_scaled = scaler.transform(X_val[[i]])
    X_test_scaled = scaler.transform(X_test[[i]])

    # 將標準化後的值重新賦值給 DataFrame
    X_train[i] = X_train_scaled
    X_val[i] = X_val_scaled
    X_test[i] = X_test_scaled

  return X_train, X_val, X_test

X_train, X_val, X_test = CustomScaler(X_train, X_val, X_test)

X_train

,today_return,sentiment,^DJI,^GSPC,^NDX,^IXIC,^SOX
0,0.564830,0.585631,0.355624,0.377236,0.525722,0.515721,0.591670
1,0.442231,0.545035,0.557596,0.578553,0.424253,0.436452,0.374338
2,0.535075,0.551635,0.359087,0.404126,0.580837,0.558035,0.383766
3,0.400162,0.561947,0.258224,0.251419,0.381444,0.380842,0.388167
4,0.318206,0.545467,0.411837,0.380522,0.324291,0.299087,0.293563
...,...,...,...,...,...,...,...
598,0.497355,0.573301,0.552968,0.534407,0.476651,0.485527,0.464971
599,0.502404,0.946136,0.617738,0.545086,0.469628,0.481824,0.452541
600,0.434839,0.633477,0.437380,0.484541,0.427071,0.432268,0.478096
601,0.688021,0.606886,0.617488,0.660028,0.668223,0.653762,0.660923


## (6) Check number

In [51]:
def calculate_class_stats(y):
    class_counts = y.value_counts()
    total_samples = len(y)
    class_ratios = class_counts / total_samples
    return class_counts, class_ratios

# 計算類別數量和比例
train_class_counts, train_class_ratios = calculate_class_stats(y_train)
val_class_counts, val_class_ratios = calculate_class_stats(y_val)
test_class_counts, test_class_ratios = calculate_class_stats(y_test)

# 創建包含數量和比例的 DataFrame
class_stats = pd.DataFrame({
    'Train Count': train_class_counts,
    'Train Ratio': train_class_ratios,
    'Validation Count': val_class_counts,
    'Validation Ratio': val_class_ratios,
    'Test Count': test_class_counts,
    'Test Ratio': test_class_ratios
})

# 打印 DataFrame
print(class_stats)


   Train Count  Train Ratio  Validation Count  Validation Ratio  Test Count  \
1          322     0.533997               113          0.562189         114   
0          281     0.466003                88          0.437811          88   

   Test Ratio  
1    0.564356  
0    0.435644  


In [52]:
# Time period
print('Time Period')
print('From:', time_start)
print('To:', time_end, '\n')

# Sample size
print('Sample size:', X.shape[0])
print('Feature:', X.columns, '\n')
print('Target:', y.name, '\n')
print('Train: Val: Test =', X_train.shape[0]/X.shape[0], X_test.shape[0]/X.shape[0], X_val.shape[0]/X.shape[0],
      X_train.shape[0], X_test.shape[0], X_val.shape[0])

Time Period
From: 2016-01-01T00:00:00
To: 2019-12-31T00:00:00 

Sample size: 1006
Feature: Index(['today_return', 'sentiment', '^DJI', '^GSPC', '^NDX', '^IXIC', '^SOX'], dtype='object') 

Target: excess_return_^GSPC_cate 

Train: Val: Test = 0.5994035785288271 0.20079522862823063 0.19980119284294234 603 202 201


## Benchmark: Random Guess

In [53]:
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

1. Fix random seed

In [54]:
import numpy as np
from sklearn.metrics import accuracy_score

np.random.seed(config['seed'])
def random_guess(text, true_label):
  random_predictions = np.random.randint(2, size=len(true_label))  # 随机猜测100个0和1的预测
  accuracy = accuracy_score(true_label, random_predictions)
  conf_matrix = confusion_matrix(true_label, random_predictions)
  report = classification_report(true_label, random_predictions)
  print(f"{text} acc：", accuracy)
  print(conf_matrix)
  print(report)


random_guess('Train', y_train)
random_guess('Val', y_val)
random_guess('Test', y_test)

Train acc： 0.4461028192371476
[[118 163]
 [171 151]]
              precision    recall  f1-score   support

           0       0.41      0.42      0.41       281
           1       0.48      0.47      0.47       322

    accuracy                           0.45       603
   macro avg       0.44      0.44      0.44       603
weighted avg       0.45      0.45      0.45       603

Val acc： 0.48258706467661694
[[42 46]
 [58 55]]
              precision    recall  f1-score   support

           0       0.42      0.48      0.45        88
           1       0.54      0.49      0.51       113

    accuracy                           0.48       201
   macro avg       0.48      0.48      0.48       201
weighted avg       0.49      0.48      0.48       201

Test acc： 0.45544554455445546
[[42 46]
 [64 50]]
              precision    recall  f1-score   support

           0       0.40      0.48      0.43        88
           1       0.52      0.44      0.48       114

    accuracy                    

2. Average of 10000 times

In [55]:
# import numpy as np
# from sklearn.metrics import accuracy_score

# def random_guess(text, true_label):
#     accuracies = []

#     for _ in range(1000):
#         random_predictions = np.random.randint(2, size=len(true_label))
#         accuracy = accuracy_score(true_label, random_predictions)
#         accuracies.append(accuracy)

#     average_accuracy = np.mean(accuracies)

#     print(f"{text} Acc：", average_accuracy)

# random_guess('Train ', y_train)
# random_guess('Val   ', y_val)
# random_guess('Test  ', y_test)
# random_guess('Overal', y)

Train  Acc： 0.4994892205638474
Val    Acc： 0.5008258706467662
Test   Acc： 0.4989455445544555
Overal Acc： 0.500138170974155


In [56]:
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
np.random.seed(config['seed'])
def random_guess(text, true_label, num_guesses=10000):
    accuracy_list = []
    conf_matrix_sum = np.zeros((2, 2))
    report_list = []

    for _ in range(num_guesses):
        random_predictions = np.random.randint(2, size=len(true_label))
        accuracy = accuracy_score(true_label, random_predictions)
        conf_matrix = confusion_matrix(true_label, random_predictions)
        report = classification_report(true_label, random_predictions, output_dict=True)

        accuracy_list.append(accuracy)
        conf_matrix_sum += conf_matrix
        report_list.append(report)

    average_accuracy = np.mean(accuracy_list)
    average_conf_matrix = conf_matrix_sum / num_guesses

    # Calculate average precision, recall, and F1-score
    print(f"{text} Acc：")
    macro_precision = np.mean([report['macro avg']['precision'] for report in report_list])
    macro_recall = np.mean([report['macro avg']['recall'] for report in report_list])
    macro_f1 = np.mean([report['macro avg']['f1-score'] for report in report_list])

    print(f"Macro avg - Precision: {macro_precision}, Recall: {macro_recall}, F1-score: {macro_f1}")

    weighted_precision = np.mean([report['weighted avg']['precision'] for report in report_list])
    weighted_recall = np.mean([report['weighted avg']['recall'] for report in report_list])
    weighted_f1 = np.mean([report['weighted avg']['f1-score'] for report in report_list])

    print(f"weighted avg - Precision: {weighted_precision}, Recall: {weighted_recall}, F1-score: {weighted_f1}")


    # avg_precision_0 = np.mean([report['0']['precision'] for report in report_list])
    # avg_precision_1 = np.mean([report['1']['precision'] for report in report_list])
    # avg_recall_0 = np.mean([report['0']['recall'] for report in report_list])
    # avg_recall_1 = np.mean([report['1']['recall'] for report in report_list])
    # avg_f1_0 = np.mean([report['0']['f1-score'] for report in report_list])
    # avg_f1_1 = np.mean([report['1']['f1-score'] for report in report_list])

    # Print the average results
    # print(f"{text} Average Accuracy: {average_accuracy}")
    # print("Average Confusion Matrix:")
    # print(average_conf_matrix)
    # print("Average Classification Report:")
    # print(f"Class 0 - Precision: {avg_precision_0}, Recall: {avg_recall_0}, F1-score: {avg_f1_0}")
    # print(f"Class 1 - Precision: {avg_precision_1}, Recall: {avg_recall_1}, F1-score: {avg_f1_1}")

random_guess('Train ', y_train)
random_guess('Val   ', y_val)
random_guess('Test  ', y_test)

Train  Acc：
Macro avg - Precision: 0.49991808831982787, Recall: 0.4999183108242523, F1-score: 0.4991376456552637
weighted avg - Precision: 0.5022291987422405, Recall: 0.4999179104477612, F1-score: 0.5002938767369545
Val    Acc：
Macro avg - Precision: 0.5003387207037615, Recall: 0.5003453288415125, F1-score: 0.49778530521569136
weighted avg - Precision: 0.5080755849975575, Recall: 0.5003646766169155, F1-score: 0.5016735572043262
Test   Acc：
Macro avg - Precision: 0.500081973717605, Recall: 0.5000811004784689, F1-score: 0.4973839755640751
weighted avg - Precision: 0.5083738470884211, Recall: 0.5000886138613861, F1-score: 0.5015457274389463
